In [142]:
import nglview as nv
import MDAnalysis as mda
import MDAnalysis.analysis.align
import pandas as pd
import BioSimSpace as bss

In [2]:
u = mda.Universe("../inputs/protein/vim2.gromacs.pdb")
lig = u.select_atoms("resname PT1")
v = nv.show_mdanalysis(lig)
v

NGLWidget()

In [121]:
with open("../../md_vim2/outputs/parmed/pt1_dry.out", "r") as file:
    lines = file.readlines()

for i in range(len(lines)):
    if "mask" in lines[i]:
        mask_line = i

info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
df = pd.DataFrame(info[1:], columns=column_headers)
net_charge_pt1 = df["Charge"].astype(float).sum()


-0.7112999999999998


In [112]:
with open("../../md_vim2/outputs/pt1_parmed.out", "r") as file:
    lines = file.readlines()

for i in range(len(lines)):
    if "mask" in lines[i]:
        mask_line = i

info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
df = pd.DataFrame(info[1:], columns=column_headers)
net_charge_pt1 = df["Charge"].astype(float).sum()
# net_charge_pt1
print(df)

    ATOM  RES RESNAME  NAME TYPE At.#  Radius   Depth     Mass   Charge  \
0   3418  233     PT1   C02    c    6  1.8606  0.0988  12.0100   0.8294   
1   3419  233     PT1   C03   cc    6  1.8606  0.0988  12.0100  -0.4941   
2   3420  233     PT1   C04   cd    6  1.8606  0.0988  12.0100   0.2997   
3   3421  233     PT1   C05   c3    6  1.9069  0.1078  12.0100  -0.3215   
4   3422  233     PT1   C10   ca    6  1.8606  0.0988  12.0100  -0.3040   
5   3423  233     PT1   C11   ca    6  1.8606  0.0988  12.0100   0.2951   
6   3424  233     PT1   C13   ca    6  1.8606  0.0988  12.0100  -0.4058   
7   3425  233     PT1   C14   ca    6  1.8606  0.0988  12.0100   0.4017   
8   3426  233     PT1   C16   ca    6  1.8606  0.0988  12.0100  -0.4571   
9   3427  233     PT1   C17   ca    6  1.8606  0.0988  12.0100   0.4785   
10  3428  233     PT1   F12    f    9  1.7029  0.0832  19.0000  -0.1145   
11  3429  233     PT1   F15    f    9  1.7029  0.0832  19.0000  -0.1347   
12  3430  233     PT1   O

In [97]:
with open("../../md_vim2/outputs/cs1.out", "r") as file:
    lines = file.readlines()

for i in range(len(lines)):
    if "mask" in lines[i]:
        mask_line = i

info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
df = pd.DataFrame(info[1:], columns=column_headers)
net_charge_cs1 = df["Charge"].astype(float).sum()
net_charge_cs1

-0.3157999999999999

In [106]:
zn1 = 0.9916
zn2 = 1.1071

-1.0270999999999997

In [99]:
with open("../../md_vim2/outputs/ap1.out", "r") as file:
    lines = file.readlines()

for i in range(len(lines)):
    if "mask" in lines[i]:
        mask_line = i

info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
df = pd.DataFrame(info[1:], columns=column_headers)
net_charge_ap1 = df["Charge"].astype(float).sum()
net_charge_ap1

-0.36529999999999974

In [103]:
with open("../../md_vim2/outputs/hd3.out", "r") as file:
    lines = file.readlines()

for i in range(len(lines)):
    if "mask" in lines[i]:
        mask_line = i

info = [line.split() for line in lines[mask_line:] if "mask" not in line][1:-2]
column_headers = [element for element in info[0] if element != "GB" and element != "LJ"]
df = pd.DataFrame(info[1:], columns=column_headers)
net_charge_hd3 = df["Charge"].astype(float).sum()
net_charge_hd3

0.41290000000000016

In [111]:
net_charge_cs1 + net_charge_ap1 + net_charge_hd3 + net_charge_pt1

-0.9794999999999993

In [113]:
net_charge_pt1 + zn1 + zn2

1.3874000000000002

In [114]:
net_charge_pt1

-0.7112999999999998

In [122]:
pt = bss.IO.readMolecules("../inputs/ligands/ligand_8_xtal.sdf")
pt.charge()

0.0000e+00 |e|

In [153]:
parm_u = mda.Universe("../inputs/protein/PT1.mol2")
xtal_u = mda.Universe("../inputs/ligands/ligand_8_xtal.mol2")
new_coords = xtal_u.atoms.positions
new_univers = parm_u.load_new(new_coords)
parm_u.atoms.positions

array([[25.556 , 27.158 , 39.628 ],
       [25.902 , 28.505 , 39.647 ],
       [24.892 , 29.458 , 39.672 ],
       [25.13  , 30.823 , 39.028 ],
       [23.584 , 29.058 , 40.008 ],
       [22.572 , 29.99  , 40.313 ],
       [21.296 , 29.547 , 40.639 ],
       [21.017 , 28.184 , 40.666 ],
       [22.013 , 27.265 , 40.366 ],
       [23.307 , 27.712 , 40.035 ],
       [22.84  , 31.327 , 40.288 ],
       [19.763 , 27.751 , 40.986 ],
       [26.4   , 26.334 , 39.508 ],
       [25.14  , 32.112 , 36.5881],
       [26.306 , 29.941 , 36.692 ],
       [23.842 , 30.02  , 36.724 ],
       [24.268 , 26.785 , 39.745 ],
       [25.103 , 30.719 , 37.17  ],
       [24.416 , 31.455 , 39.33  ],
       [26.02  , 31.172 , 39.322 ],
       [26.863 , 28.783 , 39.643 ],
       [20.577 , 30.208 , 40.854 ],
       [21.812 , 26.286 , 40.385 ],
       [23.031 , 30.585 , 36.911 ]], dtype=float32)

In [154]:
new_univers.atoms.positions

array([[25.556 , 27.158 , 39.628 ],
       [25.902 , 28.505 , 39.647 ],
       [24.892 , 29.458 , 39.672 ],
       [25.13  , 30.823 , 39.028 ],
       [23.584 , 29.058 , 40.008 ],
       [22.572 , 29.99  , 40.313 ],
       [21.296 , 29.547 , 40.639 ],
       [21.017 , 28.184 , 40.666 ],
       [22.013 , 27.265 , 40.366 ],
       [23.307 , 27.712 , 40.035 ],
       [22.84  , 31.327 , 40.288 ],
       [19.763 , 27.751 , 40.986 ],
       [26.4   , 26.334 , 39.508 ],
       [25.14  , 32.112 , 36.5881],
       [26.306 , 29.941 , 36.692 ],
       [23.842 , 30.02  , 36.724 ],
       [24.268 , 26.785 , 39.745 ],
       [25.103 , 30.719 , 37.17  ],
       [24.416 , 31.455 , 39.33  ],
       [26.02  , 31.172 , 39.322 ],
       [26.863 , 28.783 , 39.643 ],
       [20.577 , 30.208 , 40.854 ],
       [21.812 , 26.286 , 40.385 ],
       [23.031 , 30.585 , 36.911 ]], dtype=float32)

< Timestep 0 >
